In [1]:
# dfseer 后端 最牛逼的版本时间
# 求acc 和 model variance
import numpy as np
import pandas as pd
import json
import csv
from pymongo import MongoClient
from time import strftime
import pytz
import datetime
from collections import Counter# 很重要主要是为了给字典排序
client = MongoClient('localhost', 27017)
mydb = client['Dong_chi_2020']
# lassoedDataFromSimilarityView = {'SimilarityViewData': {'data': [{'endPeriod': 201805, 'item': 'item2356'},{'endPeriod': 201806, 'item': 'item1647'}, {'endPeriod': 201807, 'item': 'item1944'}, {'endPeriod': 201806, 'item': 'item1944'}, {'endPeriod': 201807, 'item': 'item2295'}, {'endPeriod': 201805, 'item': 'item1944'}]}}
lassoedDataFromSimilarityView = {'SimilarityViewData': {'data': [{'endPeriod': 201807, 'item': 'item2210'},{'endPeriod': 201807, 'item': 'item2283'}, {'endPeriod': 201807, 'item': 'item2317'},{'endPeriod': 201806, 'item': 'item1647'}]}}



similarityDataIndex = lassoedDataFromSimilarityView['SimilarityViewData']['data']
endPeriodDict ={'data': [{'endPeriod': similarityDataIndex[0]['endPeriod']}]}
collectionKey = {}
newEndPeriod = {}
dfItemInfo = []
predictMonth = 201807
for i in range(len(similarityDataIndex)):
    if similarityDataIndex[i]['endPeriod'] not in endPeriodDict['data']:
        newEndPeriod['endPeriod'] = similarityDataIndex[i]['endPeriod']
        endPeriodDict['data'].append(newEndPeriod)
    month = similarityDataIndex[i]['endPeriod']
    collectionKey = {'data':[{'collection': 'collection_'+str(month)}]}
    collection = mydb[collectionKey['data'][0]['collection']]
    itemResults = collection.find({'item':similarityDataIndex[i]['item']})#.sort('accuracy', -1)
    for itemResult in itemResults:
        if float(itemResult['accuracy'])>-0.01:
            itemResult['endPeriod'] = similarityDataIndex[i]['endPeriod']
            itemResult['_id'] = ''
            dfItemInfo.append(itemResult)
dfItemInfo = pd.DataFrame(dfItemInfo) #所有lasso点的信息每个点25条数据

modelName= ["GMNR", "ada", "arima", "arimax", "cat", "elnet", "extra", "gbdt", "knn", "lasso", "lgb", "lr", "myarima", "myarimax", "rf", "ridge", "svr", "xgb", "arima_ts", "exps_ts", "myarima_ts", "lstm_classic", "maml", "rl2", "rnn_classic"]
models = {'models': modelName}
ModelAcc = []
ModelVar = []
for i in range(len(modelName)):
    tmp = {}
    dfModelAcc = dfItemInfo.loc[(dfItemInfo.model == models['models'][i])&(dfItemInfo.endPeriod != predictMonth)]
    dfModelAcc['accuracy'] = pd.to_numeric(dfModelAcc['accuracy'],errors='coerce')
    acc = dfModelAcc['accuracy'].mean(axis = 0)
    var = dfModelAcc['accuracy'].var(axis = 0)
    tmp['model'] = models['models'][i]
    tmp['acc'] = acc
    tmp['variance'] = var
    ModelAcc.append(tmp)    
lassoedDataModelAccList = sorted(ModelAcc, key=lambda k: k['acc'], reverse=True)
# print(lassoedDataModelAccList)
lassoedDataModelAccDict = {'lassoedDataModelAcc':lassoedDataModelAccList}
dfLassoedDataModelAccList = pd.DataFrame(lassoedDataModelAccList)
# print(lassoedDataModelAccDict)

# 求 variance# variance 进行计算  最终版
monthArray = [201708, 201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804, 201805, 201806, 201807]
modelName= ["GMNR", "ada", "arima", "arimax", "cat", "elnet", "extra", "gbdt", "knn", "lasso", "lgb", "lr", "myarima", "myarimax", "rf", "ridge", "svr", "xgb", "arima_ts", "exps_ts", "myarima_ts", "lstm_classic", "maml", "rl2", "rnn_classic"]
models = {'models': modelName}
# 统计次数构造全0
modelRankDict = {}
for i in range(len(modelName)):
    modelRankDict[modelName[i]] = 0
similarityDataIndex = lassoedDataFromSimilarityView['SimilarityViewData']['data']
endPeriodDict ={'data': [{'endPeriod': similarityDataIndex[0]['endPeriod']}]}
collectionKey = {}
newEndPeriod = {}
modelVariance = {}
tmpVariance ={}
lassoedDataSummary = []

# numNonePredictPoint = 0
# for i in range(len(similarityDataIndex)):
#     if similarityDataIndex[i]['endPeriod'] != predictMonth:
#         numNonePredictPoint = numNonePredictPoint + 1


for i in range(len(similarityDataIndex)):
    print('=========',i,'========',similarityDataIndex[i]['item'],'=================')
    lassoPointInfo = []
    if similarityDataIndex[i]['endPeriod'] not in endPeriodDict['data']:
        newEndPeriod['endPeriod'] = similarityDataIndex[i]['endPeriod']
        endPeriodDict['data'].append(newEndPeriod)
    month = similarityDataIndex[i]['endPeriod']
    collectionKey = {'data':[{'collection': 'collection_'+str(month)}]}
    collection = mydb[collectionKey['data'][0]['collection']]
    lassoPointResults = collection.find({'item':similarityDataIndex[i]['item']})#.sort('accuracy', -1)
    for lassoPointResult in lassoPointResults:
        lassoPointResult['endPeriod'] = similarityDataIndex[i]['endPeriod']
        if float(lassoPointResult['accuracy'])>-0.01:
            # print(lassoPointResult['accuracy'])
            lassoPointInfo.append(lassoPointResult)
    if len(lassoPointInfo)!= 0:
        dfLassoPointInfo = pd.DataFrame(lassoPointInfo)           
        modelItemVar = []
        # print('-********',dfLassoPointInfo['item'],'********')

        for j in range(dfLassoPointInfo.shape[0]):
            endPeriod = dfLassoPointInfo.iloc[j]['endPeriod']
            monthIndex = monthArray.index(endPeriod)
            tmpAcc = []
            for p in range(10):
                currentMonth = monthArray[monthIndex-p]
                collectionKey = {'data':[{'collection': 'collection_'+str(currentMonth)}]}
                collection = mydb[collectionKey['data'][0]['collection']]
                eachMonthResults= collection.find({'$and':[{'item':dfLassoPointInfo.iloc[j]['item']},{'model':dfLassoPointInfo.iloc[j]['model']}]})
                eachCollectionAcc = []
                if eachMonthResults.count()!=0:
                    for eachMonthResult in eachMonthResults:
                        if float(eachMonthResult['accuracy'])>-0.01:
                            # print(eachMonthResult['accuracy'])
                            eachCollectionAcc.append(eachMonthResult['accuracy'])
                    tmp =np.array(eachCollectionAcc)
                    tmp = tmp.astype(float)
                    eachMonthMeanAcc = np.nanmean(tmp)
                    tmpAcc.append(eachMonthMeanAcc)
            eachLassoedPointModelItemVar = np.nanvar((np.array(tmpAcc)).astype(float))
            modelItemVar.append(eachLassoedPointModelItemVar)
            np.array(modelItemVar)
        dfLassoPointInfo.insert(7,'modelItemVar', modelItemVar)
        for m in range(dfLassoPointInfo.shape[0]):
            if i == 0:
                tmpVariance[dfLassoPointInfo.iloc[m]['model']] =[dfLassoPointInfo.iloc[m]['modelItemVar']]
            else:
                tmpVariance[dfLassoPointInfo.iloc[m]['model']].append(dfLassoPointInfo.iloc[m]['modelItemVar'])  
        # print('+++++variance++++',modelItemVar )
        
        
#         -------------------------------------------------------------------------------------------
        
        
        dfLassoPointInfoDescending = dfLassoPointInfo.sort_values(by='accuracy', ascending=False)
        lassoPointInfoDescendingDict = dfLassoPointInfoDescending.to_dict(orient='records')
        
        
        if lassoPointInfoDescendingDict[0]['endPeriod'] != predictMonth:
                    
            for q in range(5):
                tmpModel = dfLassoPointInfoDescending.iloc[q]['model']
                modelRankDict[tmpModel]=modelRankDict[tmpModel]+1
        # lassoPointInfoDescending
        dfLassoPointInfoDescending= dfLassoPointInfoDescending.loc[:,['item', 'endPeriod', 'model','accuracy', 'modelItemVar', 'predictValue', 'realValue' ]]
        lassoPointInfoDescendingDict = dfLassoPointInfoDescending.to_dict(orient='records')
        # print(lassoPointInfoDescendingDict)
        tmpLassoPointInfoDescendingDict = {}
        tmpLassoPointInfoDescendingDict['item'] = lassoPointInfoDescendingDict[0]['item']
        tmpLassoPointInfoDescendingDict['endPeriod'] = lassoPointInfoDescendingDict[0]['endPeriod']
        for w in range(len(lassoPointInfoDescendingDict)):
            if w == 0:                
                keys = ('model','accuracy', 'modelItemVar', 'predictValue', 'realValue', 'endPeriod', 'item')
                tmpLassoPointInfoModel =  {k: lassoPointInfoDescendingDict[w][k] for k in keys}
                tmpLassoPointInfoDescendingDict['model'] = [tmpLassoPointInfoModel]
            else:
                keys = ('model','accuracy', 'modelItemVar', 'predictValue', 'realValue', 'endPeriod', 'item')
                tmpLassoPointInfoModel =  {k: lassoPointInfoDescendingDict[w][k] for k in keys} 
                if w%2 ==0:
                    tmpLassoPointInfoDescendingDict['model'].append(tmpLassoPointInfoModel)
        lassoedDataSummary.append(tmpLassoPointInfoDescendingDict)    

        # print(lassoPointInfoDescendingDict)
modelRankDict=dict(Counter(modelRankDict).most_common())         
print(modelRankDict)  
tmpmaxModelVarianceList = []
for i in range(len(lassoedDataModelAccDict['lassoedDataModelAcc'])):
    tmpmaxModelVarianceList.append(lassoedDataModelAccDict['lassoedDataModelAcc'][i]['variance'])
maxModelVariance = max(tmpmaxModelVarianceList)
# print('+++++',modelVariance,'++++')
modelInfo = []
for i in range( len(modelName)):
    tmpModelInfo = {}
    tmpModelInfo['modelName'] = modelName[i]
    tmp = dfLassoedDataModelAccList.loc[(dfLassoedDataModelAccList['model']==modelName[i]),['acc']].values
    tmpVarianceLast = dfLassoedDataModelAccList.loc[(dfLassoedDataModelAccList['model']==modelName[i]),['variance']].values
    tmpModelInfo['accuracy'] = tmp[0][0]
    tmpModelInfo['variance'] = tmpVarianceLast[0][0]/maxModelVariance
    tmpModelInfo['topRank'] = (modelRankDict[modelName[i]])/max(modelRankDict.values())
    modelInfo.append(tmpModelInfo)
# print(modelInfo)
# print('----------分割线-------------')
# print(lassoedDataSummary)
result = {'modelInformation': modelInfo, 'lassoedDataInformation': lassoedDataSummary}
print('------------------------------------', result, '---------------')
# print('哈哈哈',lassoedDataSummary)
# print(lassoedDataSummary)

tz = pytz.timezone('Asia/Shanghai')
currentTime = datetime.datetime.now(tz).strftime("%Y%m%d_%H%M%S")
jsonData = json.dumps(lassoedDataSummary)
fileObject = open('lassoedPoints_'+currentTime+'.json', 'w')
fileObject.write(jsonData)
fileObject.close()
print('finish')








C:\Users\dsunae\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


========= 0 ======== item2210 =================


C:\Users\dsunae\Anaconda3\envs\py35\lib\site-packages\ipykernel_launcher.py:112: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


========= 1 ======== item2283 =================
========= 2 ======== item2317 =================
========= 3 ======== item1647 =================
{'knn': 0, 'lasso': 0, 'maml': 0, 'myarima': 0, 'exps_ts': 0, 'rl2': 0, 'myarimax': 1, 'arimax': 1, 'lr': 0, 'ada': 0, 'extra': 0, 'lgb': 1, 'rf': 0, 'GMNR': 0, 'myarima_ts': 0, 'rnn_classic': 0, 'xgb': 0, 'svr': 0, 'arima': 1, 'ridge': 0, 'gbdt': 1, 'cat': 0, 'arima_ts': 0, 'lstm_classic': 0, 'elnet': 0}
------------------------------------ {'lassoedDataInformation': [{'endPeriod': 201807, 'model': [{'modelItemVar': 0.05527136112222223, 'realValue': '0.006966666666666666', 'endPeriod': 201807, 'model': 'myarima', 'accuracy': '0.82', 'predictValue': '0.008188639978545341', 'item': 'item2210'}, {'modelItemVar': 0.07961182543333332, 'realValue': '0.006966666666666666', 'endPeriod': 201807, 'model': 'extra', 'accuracy': '0.33083333333333337', 'predictValue': '0.10274666666666664', 'item': 'item2210'}, {'modelItemVar': 0.07658657515555556, 'realVal